In [1]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import requests
import json


# --------------------------------Get daily stock data from yahoo finance api----------------------------------
# Parameter: stock ticker symbol, start date of the data, end date of the data

def getDailyData(stock_symbol, start_date, end_date):
    data = yf.download(stock_symbol, start=start_date, end=end_date)
    return data


# ----------------------------------------Convert sentimental date---------------------------------------------
def convert_senti_Date_string(dateStr): #Input:20200901T0930 --> Output:2020-09-01

    # Use strptime to parse the input date string
    datetime_obj = datetime.strptime(date_string, '%Y%m%dT%H%M%S')
    
    # Format the datetime object as 'YYYY-MM-DD'
    formatted_date = datetime_obj.strftime('%Y-%m-%d')
    
    return formatted_date


def convertDT_SentimentalDT(date_str, time_str): #Input: Date=2020-09-01 & Time:9:30 --> Output:20200901T0930
    # Parse the input date and time strings
    date_obj = datetime.strptime(date_str, "%Y-%m-%d")
    time_obj = datetime.strptime(time_str, "%H:%M")

    # Format the date and time in the desired format
    sentimental_dt_str = date_obj.strftime("%Y%m%dT") + time_obj.strftime("%H%M")
    return sentimental_dt_str


#---------------------------------------------convert date time--------------------------------------------------------
def convertStrToDatetime(t): # Input: Date(String)--> Output: Date(datetime)
    dateFormat = "%Y-%m-%d" 
    dateTimeObj = datetime.strptime(t, dateFormat)
    return dateTimeObj

def transform_datetime(input_date):
    try:
        # Parse the input date string into a datetime object
        date_obj = datetime.strptime(input_date, "%Y%m%dT%H%M%S")

        # Format the datetime object as a string in the desired format
        formatted_date = date_obj.strftime("%Y-%m-%d")

        return formatted_date
    except ValueError:
        return "Invalid input date format"
        
#***------------Getting exponential moving average (EMA) values-------------------   
def get_ema(ss, interval):

    apikey = "487EQ5T70C156H7X"
    url = f"https://www.alphavantage.co/query?function=EMA&symbol={ss}&interval={interval}&time_period=200&series_type=open&apikey={apikey}"
    r = requests.get(url)
    data = r.json()

    symbol_list = []
    date_list = []
    ema_list = []

    symbol = data["Meta Data"]["1: Symbol"]
    ema_data = data["Technical Analysis: EMA"]

    for ema in ema_data.items():
        symbol_list.append(str.upper(symbol))
        date_list.append(convertStrToDatetime(ema[0]))
        ema_list.append(ema[1]["EMA"])

    EMA_Dict = {
        "Dates": date_list,
        "Symbol": symbol_list,
        "EMA": ema_list
    }

    EMA_DF = pd.DataFrame(EMA_Dict)
    
    return EMA_DF
#***---------------------------------Getting  relative strength index (RSI) values----------------------------------
def get_rsi(ss, interval):
    apikey = "487EQ5T70C156H7X"
    url = f"https://www.alphavantage.co/query?function=RSI&symbol={ss}&interval={interval}&time_period=200&series_type=open&apikey={apikey}"
    r = requests.get(url)
    data = r.json()

    symbol_list = []
    date_list = []
    rsi_list = []
    
    symbol = data["Meta Data"]["1: Symbol"]
    rsi_data = data["Technical Analysis: RSI"]

    for rsi in rsi_data.items():
        symbol_list.append(str.upper(symbol))
        date_list.append(convertStrToDatetime(rsi[0]))
        rsi_list.append(rsi[1]["RSI"])

    RSI_Dict = {
        "Dates": date_list,
        "Symbol": symbol_list,
        "RSI": rsi_list
    }
    RSI_DF = pd.DataFrame(RSI_Dict)
    return RSI_DF










#------------------------------Main-------------------------------------
# Define company symbols and other variables
company = ["MSFT", "AAPL", "GOOG","META","IBM","TSLA","AMZN"]
start_date = "2023-09-01"
# current_date = datetime.now()
dt_start_date = convertStrToDatetime(start_date)
current_date = convertStrToDatetime("2023-09-28")
time = "9:30"

# Create an empty list to store dataframes
dataframes_with_company = []

# Collect data for each company and add it to the list with a 'Company' column
for stock_symbol in company:
    dt_start = dt_start_date
    while dt_start <= current_date:
        start_date_str = dt_start.strftime("%Y-%m-%d")
        stock_data = getDailyData(stock_symbol, start_date, start_date_str)
        stock_data['Company'] = str.upper(stock_symbol)  # Add a 'Company' column in upper case
        dataframes_with_company.append(stock_data)
        dt_start += timedelta(days=1)# Increment the day by one

# Concatenate the dataframes
combined_df = pd.concat(dataframes_with_company, axis=0)
# Reset the index of the resulting concatenated dataframe
combined_df.reset_index(inplace=True)

# modifying data for machine learning
for stock in company:
    combined_df[f"{str.upper(stock)}"]=0  # Create columns with each company name and give it value 0
    for index, row in combined_df.iterrows():
        for stock in company:
            if str.upper(row['Company']) == str.upper(stock):  # if company name match the column name 
                combined_df.at[index, str.upper(stock)] = 1    # if yes give it a value of 1
    
# Display the concatenated dataframe for stockdata
combined_df





#------------------------------------------------Getting EMA data-----------------------------------------
all_ema_data = []  # Create a list to store EMA DataFrames
interval = "daily"
for stock in company:
    EMA_DF = get_ema(stock, interval)
    all_ema_data.append(EMA_DF)  # Append each stock's DataFrame to the list
# Concatenate all DataFrames in the list
final_ema_df = pd.concat(all_ema_data, ignore_index=True)
final_ema_df



#-----------------------------------------------Getting RSI Data---------------------------------------------
interval = "daily"
all_rsi_data = []  # Create a list to store RSI DataFrames

for stock in company:
    RSI_DF = get_rsi(stock, interval)
    all_rsi_data.append(RSI_DF)  # Append each stock's DataFrame to the list

# Concatenate all DataFrames in the list
final_rsi_df = pd.concat(all_rsi_data, ignore_index=True)
final_rsi_df



#----------------------------------- merge rsi and ema dataframe on dates and symbol---------------------------
rsi_ema_df = final_rsi_df.merge(final_ema_df, on=['Dates', 'Symbol'], how='inner')
rsi_ema_df

#-------------------------------------Creating final DataFrame for download----------------------------
#Change the column name to match the other 
stock_df = combined_df.rename(columns={"Date": "Dates","Company":"Symbol"})

merged_df = pd.merge(stock_df, rsi_ema_df, on=["Dates", "Symbol"], how="inner")
merged_df




[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['MSFT']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-09-01 -> 2023-09-01)')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******


1 Failed download:
['AAPL']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-09-01 -> 2023-09-01)')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******


1 Failed download:
['AMZN']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-09-01 -> 2023-09-01)')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

/tmp/ipykernel_10139/603376562.py:144: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  combined_df = pd.concat(dataframes_with_company, axis=0)


,Dates,Open,High,Low,Close,Adj Close,Volume,Symbol,MSFT,AAPL,GOOG,META,IBM,TSLA,AMZN,RSI,EMA
0,2023-09-01,331.309998,331.989990,326.779999,328.660004,328.660004,14931200.0,MSFT,1,0,0,0,0,0,0,53.1872,300.3726
1,2023-09-01,331.309998,331.989990,326.779999,328.660004,328.660004,14931200.0,MSFT,1,0,0,0,0,0,0,53.1872,300.3726
2,2023-09-01,331.309998,331.989990,326.779999,328.660004,328.660004,14931200.0,MSFT,1,0,0,0,0,0,0,53.1872,300.3726
3,2023-09-01,331.309998,331.989990,326.779999,328.660004,328.660004,14931200.0,MSFT,1,0,0,0,0,0,0,53.1872,300.3726
4,2023-09-01,331.309998,331.989990,326.779999,328.660004,328.660004,14931200.0,MSFT,1,0,0,0,0,0,0,53.1872,300.3726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633,2023-09-25,129.360001,131.779999,128.770004,131.270004,131.270004,46017800.0,AMZN,0,0,0,0,0,0,1,50.6942,122.0211
1634,2023-09-25,129.360001,131.779999,128.770004,131.270004,131.270004,46017800.0,AMZN,0,0,0,0,0,0,1,50.6942,122.0211
1635,2023-09-26,130.119995,130.389999,125.279999,125.980003,125.980003,73048200.0,AMZN,0,0,0,0,0,0,1,50.7807,122.1017
1636,2023-09-26,130.119995,130.389999,125.279999,125.980003,125.980003,73048200.0,AMZN,0,0,0,0,0,0,1,50.7807,122.1017


In [4]:
merged_df.to_csv('merged_data.csv', index=False)

In [5]:
import boto3
# Let's use Amazon S3
s3 = boto3.resource('s3')

# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

with open('Stock_Data(20203-09-28).csv', 'rb') as data:
    s3.Bucket('stockprediction-wg').put_object(Key='test.jpg', Body=data)

stockprediction-wg
